### Create a matrix of size num_models x num_folds containing normalized loglikelihood for both train and test splits

In [1]:
"""
Imports
"""
import json
import sys
import numpy as np
import os

from post_processing_utils import load_data, load_session_fold_lookup, \
    prepare_data_for_cv, calculate_baseline_test_ll, \
    calculate_glm_test_loglikelihood, calculate_cv_bit_trial, \
    return_glmhmm_nll, return_lapse_nll

In [2]:
"""
Parameters
"""
# Parameters
C = 2  # number of output classes
num_folds = 5  # number of folds
D = 1  # number of output dimensions
K_max = 5  # maximum number of latent states
num_models = K_max + 2  # model for each latent + 2 lapse models
bin_num = 10

In [3]:
"""
DIRECTORIES
"""
data_dir = '../../data/ibl/data_for_cluster/'
data_dir = '/home/ines/repositories/representation_learning_variability/DATA/GLMHMM/'
#data_dir = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/GLMHMM/'

"""
Create folders to save processed data
"""
# Create directories for saving data:
processed_ibl_data_path = data_dir + "data_for_cluster/" + "data_by_animal/"

# Create directory for results:
results_dir = '../../../results/proficient/ibl_global_fit/'


In [4]:

# Load data
inpt, y, session = load_data(data_dir + "data_for_cluster/" + 'all_animals_concat.npz')
session_fold_lookup_table = load_session_fold_lookup(
    data_dir + "data_for_cluster/" + 'all_animals_concat_session_fold_lookup.npz')

# For later saving the data
animal_preferred_model_dict = {}
models = ["GLM", "Lapse_Model", "GLM_HMM"]

cvbt_folds_model = np.zeros((num_models, num_folds))
cvbt_train_folds_model = np.zeros((num_models, num_folds))

# Save best initialization for each model-fold combination
best_init_cvbt_dict = {}
for fold in range(num_folds):
    test_inpt, test_y, test_nonviolation_mask, this_test_session, \
    train_inpt, train_y, train_nonviolation_mask, this_train_session, M,\
    n_test, n_train = prepare_data_for_cv(
        inpt, y, session, session_fold_lookup_table, fold)
    
    ll0 = calculate_baseline_test_ll(
        train_y[train_nonviolation_mask == 1, :],
        test_y[test_nonviolation_mask == 1, :], C)
    ll0_train = calculate_baseline_test_ll(
        train_y[train_nonviolation_mask == 1, :],
        train_y[train_nonviolation_mask == 1, :], C)
        
    for model in models:
        print("model = " + str(model))
        if model == "GLM":
            # Load parameters and instantiate a new GLM object with
            # these parameters
            glm_weights_file = results_dir + '/GLM/fold_' + str(
                fold) + '/variables_of_interest_iter_0.npz'

            ll_glm = calculate_glm_test_loglikelihood(
                glm_weights_file, test_y[test_nonviolation_mask == 1, :],
                test_inpt[test_nonviolation_mask == 1, :], M, C)
            ll_glm_train = calculate_glm_test_loglikelihood(
                glm_weights_file, train_y[train_nonviolation_mask == 1, :],
                train_inpt[train_nonviolation_mask == 1, :], M, C)
            cvbt_folds_model[0, fold] = calculate_cv_bit_trial(
                ll_glm, ll0, n_test)
            cvbt_train_folds_model[0, fold] = calculate_cv_bit_trial(
                ll_glm_train, ll0_train, n_train)
                
# Save cvbt_folds_model as numpy array for easy parsing across all
# models and folds
np.savez(results_dir + "/cvbt_folds_model.npz", cvbt_folds_model)
np.savez(results_dir + "/cvbt_train_folds_model.npz",
            cvbt_train_folds_model)


model = GLM
model = Lapse_Model
model = GLM_HMM
model = GLM
model = Lapse_Model
model = GLM_HMM
model = GLM
model = Lapse_Model
model = GLM_HMM
model = GLM
model = Lapse_Model
model = GLM_HMM
model = GLM
model = Lapse_Model
model = GLM_HMM
